In [8]:
import sys
sys.path.append('..')
sys.path.append('/Users/mariana/Documents/research/xlstm-jax')
sys.path.append('/Users/mariana/Documents/research/nanoTempoPFN')
sys.path.append('/Users/mariana/Documents/research/xlstm-jax')
import jax
import jax.numpy as jnp
from pathlib import Path

from src.data.containers import ShardedDataset, DataLoader
from src.data.time_features import compute_batch_time_features
from src.model.models import xLSTMTabModelWithPos, xLSTMTabModelConfig
from xlstm_jax.models.xlstm_clean.xlstm_block_stack import xLSTMBlockStack, xLSTMBlockStackConfig
from xlstm_jax.models.xlstm_clean.blocks.mlstm.layer import mLSTMLayerConfig
from xlstm_jax.models.xlstm_clean.components.feedforward import FeedForwardConfig
from xlstm_jax.models.xlstm_clean.blocks.mlstm.block import xLSTMBlockConfig

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Load a batch
dataset = ShardedDataset(root_path=Path('../data'))
loader = DataLoader(dataset, batch_size=4, future_length=64)

batch = next(iter(loader))
print(f"history: {batch.history.shape}, future: {batch.future.shape}")


history: (4, 1984, 1, 1), future: (4, 64, 1, 1)


In [4]:
# Compute time features
t_hist, t_future = compute_batch_time_features(
    start=batch.start,
    history_length=batch.history_length,
    future_length=batch.future_length,
    batch_size=batch.batch_size,
    frequency=batch.frequency,
)
print(f"t_hist: {t_hist.shape}, t_future: {t_future.shape}")


t_hist: (4, 1984, 6), t_future: (4, 64, 6)


In [ ]:

quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

config = xLSTMTabModelConfig(
            embedding_dim=16,
            num_blocks=2,
            output_dim=len(quantiles),
            context_length=32,
            tie_weights=False,
            add_embedding_dropout=False,
            add_post_blocks_norm=True,
            dtype="bfloat16",
            mlstm_block=xLSTMBlockConfig(
                mlstm=mLSTMLayerConfig(
                    conv1d_kernel_size=4,
                    qkv_proj_blocksize=4,
                    num_heads=2,
                    proj_factor=2.0,
                    embedding_dim=16,
                    bias=True,
                    dropout=0.1,
                    context_length=32,
                    dtype="bfloat16",
                ),
                _num_blocks=1,
                _block_idx=0,
                feedforward=FeedForwardConfig(
                    proj_factor=4.0,
                    embedding_dim=16,
                    dropout=0.1,
                    dtype="bfloat16",
                ),
            ),
        )
model = xLSTMTabModelWithPos(config)

x_hist = jnp.array(batch.history)  # (B, h_len, n_channels, 1)

key = jax.random.PRNGKey(0)
params = model.init(key, x_hist, t_hist, t_future)
output = model.apply(params, x_hist, t_hist, t_future)
print(f"output: {output.shape}")


AssertionError: lower_triangular_matrix must have shape (S, S), got (32, 32)

In [6]:
output

NameError: name 'output' is not defined